In [40]:
!pip install folium
!pip install alphashape

from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from shapely.geometry import  Polygon, Point, mapping, MultiPoint
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import pandas as pd
import seaborn as sns
import folium
import alphashape


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 한번에 시각화

In [41]:
# 파일 경로 설정
base_path = '/content/drive/MyDrive/bigdata/dataset/dbscan 완료/'
base_path_dot = '/content/drive/MyDrive/bigdata/dataset/전처리 완료/'

# 파일 불러오기
drunken_file=f"{base_path}취객 dbscan.geojson"
foot_traffic_file = f"{base_path}유동인구 dbscan.geojson"
illegal_parking_file=f"{base_path}상습불법주정차 dbscan.geojson"
old_architecture_file = f"{base_path}노후주택 dbscan.geojson"
cctv_file = f"{base_path_dot}cctv 데이터.csv"
security_bell_file = f"{base_path_dot}안전비상벨.csv"
light_file = f"{base_path_dot}보안등 데이터.csv"

# 데이터셋 생성
drunken_gdf = gpd.read_file(drunken_file)
foot_traffic_gdf = gpd.read_file(foot_traffic_file)
illegal_parking_gdf = gpd.read_file(illegal_parking_file)
old_architecture_gdf = gpd.read_file(old_architecture_file)
cctv_gdf = pd.read_csv(cctv_file)
security_bell_gdf = pd.read_csv(security_bell_file)
light_gdf = pd.read_csv(light_file)

# 모든 지오메트리를 Polygon으로 변환 (필요시)
drunken_gdf.geometry = drunken_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)
foot_traffic_gdf.geometry = foot_traffic_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)
illegal_parking_gdf.geometry = illegal_parking_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)
old_architecture_gdf.geometry = old_architecture_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)

# 지도 생성
m = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# Polygon만 지도에 추가하는 함수 정의
def add_hulls_to_map(gdf, color, label):
    for _, row in gdf[gdf.geometry.geom_type == "Polygon"].iterrows():
        hull_coords = [(coord[1], coord[0]) for coord in row.geometry.exterior.coords]
        folium.Polygon(
            locations=hull_coords,
            color=color,
            fill=True,
            fill_opacity=0.2,
            popup=label,
        ).add_to(m)

# 각 데이터셋의 Concave Hull 추가
add_hulls_to_map(drunken_gdf, color="blue", label="store like karaoke selling alchohol")
add_hulls_to_map(foot_traffic_gdf, color="green", label="low foot traffic")
add_hulls_to_map(illegal_parking_gdf, color="red", label="illegal_parking")
add_hulls_to_map(old_architecture_gdf, color="brown", label="old_architecture")

# 데이터프레임과 관련 정보 리스트
datasets = [
    {'data': cctv_gdf, 'color': 'purple', 'label': 'CCTV'},
    {'data': security_bell_gdf, 'color': 'grey', 'label': '보안벨'},
    {'data': light_gdf, 'color': 'black', 'label': '가로등'}
]

# 반복문으로 데이터 추가
for dataset in datasets:
    for _, row in dataset['data'].iterrows():
        folium.CircleMarker(
            location=[row['y'], row['x']],
            radius=5,
            color=dataset['color'],
            fill=True,
            fill_color=dataset['color'],
            fill_opacity=0.6,
            popup=f"{dataset['label']}: {row.get('name', '정보 없음')}"
        ).add_to(m)



# 한국외국어대학교 정문 위도와 경도
hufs_lat = 37.596584  # 위도
hufs_lon = 127.060173  # 경도

# 기존 지도에 원 추가
folium.Circle(
    location=[hufs_lat, hufs_lon],  # 위도와 경도 위치
    radius=1000,  # 원의 반경 (단위: 미터)
    color="orange",  # 원 테두리 색상
    fill=True,  # 원 채우기
    fill_color="orange",  # 채우기 색상
    fill_opacity=0.2,  # 채우기 투명도
    popup="한국외국어대학교 정문",  # 팝업 표시 텍스트
).add_to(m)

# 지도 저장 및 표시
m.save("convex_hull_map.html")
m

Output hidden; open in https://colab.research.google.com to view.

## 차례대로 시각화

In [42]:
# 파일 경로 설정
base_path = '/content/drive/MyDrive/bigdata/dataset/dbscan 완료/'
base_path_dot = '/content/drive/MyDrive/bigdata/dataset/전처리 완료/'

# 파일 불러오기
drunken_file=f"{base_path}취객 dbscan.geojson"
foot_traffic_file = f"{base_path}유동인구 dbscan.geojson"
illegal_parking_file=f"{base_path}상습불법주정차 dbscan.geojson"
old_architecture_file = f"{base_path}노후주택 dbscan.geojson"
cctv_file = f"{base_path_dot}cctv 데이터.csv"
security_bell_file = f"{base_path_dot}안전비상벨.csv"
light_file = f"{base_path_dot}보안등 데이터.csv"

# 데이터셋 생성
drunken_gdf = gpd.read_file(drunken_file)
foot_traffic_gdf = gpd.read_file(foot_traffic_file)
illegal_parking_gdf = gpd.read_file(illegal_parking_file)
old_architecture_gdf = gpd.read_file(old_architecture_file)
cctv_gdf = pd.read_csv(cctv_file)
security_bell_gdf = pd.read_csv(security_bell_file)
light_gdf = pd.read_csv(light_file)

# 모든 지오메트리를 Polygon으로 변환 (필요시)
drunken_gdf.geometry = drunken_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)
foot_traffic_gdf.geometry = foot_traffic_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)
illegal_parking_gdf.geometry = illegal_parking_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)
old_architecture_gdf.geometry = old_architecture_gdf.geometry.apply(
    lambda geom: geom if geom.geom_type == "Polygon" else geom.convex_hull
)

# 지도 생성
m = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# Polygon만 지도에 추가하는 함수 정의
def add_hulls_to_map(gdf, color, label):
    for _, row in gdf[gdf.geometry.geom_type == "Polygon"].iterrows():
        hull_coords = [(coord[1], coord[0]) for coord in row.geometry.exterior.coords]
        folium.Polygon(
            locations=hull_coords,
            color=color,
            fill=True,
            fill_opacity=0.2,
            popup=label,
        ).add_to(m)

# 한국외국어대학교 정문 위도와 경도
hufs_lat = 37.596584  # 위도
hufs_lon = 127.060173  # 경도

# 기존 지도에 원 추가
folium.Circle(
    location=[hufs_lat, hufs_lon],  # 위도와 경도 위치
    radius=1000,  # 원의 반경 (단위: 미터)
    color="orange",  # 원 테두리 색상
    fill=True,  # 원 채우기
    fill_color="orange",  # 채우기 색상
    fill_opacity=0.2,  # 채우기 투명도
    popup="한국외국어대학교 정문",  # 팝업 표시 텍스트
).add_to(m)

# 지도 저장 및 표시
m.save("convex_hull_map.html")
m

In [43]:
# 각 데이터셋의 Concave Hull 추가
add_hulls_to_map(drunken_gdf, color="blue", label="store like karaoke selling alchohol")
m

In [44]:
add_hulls_to_map(foot_traffic_gdf, color="green", label="low foot traffic")
m

In [45]:
add_hulls_to_map(illegal_parking_gdf, color="red", label="illegal_parking")
m

In [46]:
add_hulls_to_map(old_architecture_gdf, color="brown", label="old_architecture")
m

In [47]:
# 데이터프레임과 관련 정보 리스트
datasets = [
    {'data': cctv_gdf, 'color': 'purple', 'label': 'CCTV'}
]

# 반복문으로 데이터 추가
for dataset in datasets:
    for _, row in dataset['data'].iterrows():
        folium.CircleMarker(
            location=[row['y'], row['x']],
            radius=1,
            color=dataset['color'],
            fill=True,
            fill_color=dataset['color'],
            fill_opacity=0.2,
            popup=f"{dataset['label']}: {row.get('name', '정보 없음')}"
        ).add_to(m)

m

In [48]:
# 데이터프레임과 관련 정보 리스트
datasets = [
    {'data': security_bell_gdf, 'color': 'grey', 'label': '보안벨'}
]

# 반복문으로 데이터 추가
for dataset in datasets:
    for _, row in dataset['data'].iterrows():
        folium.CircleMarker(
            location=[row['y'], row['x']],
            radius=1,
            color=dataset['color'],
            fill=True,
            fill_color=dataset['color'],
            fill_opacity=0.2,
            popup=f"{dataset['label']}: {row.get('name', '정보 없음')}"
        ).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

In [49]:
# 데이터프레임과 관련 정보 리스트
datasets = [
    {'data': light_gdf, 'color': 'black', 'label': '가로등'}
]

# 반복문으로 데이터 추가
for dataset in datasets:
    for _, row in dataset['data'].iterrows():
        folium.CircleMarker(
            location=[row['y'], row['x']],
            radius=1,
            color=dataset['color'],
            fill=True,
            fill_color=dataset['color'],
            fill_opacity=0.2,
            popup=f"{dataset['label']}: {row.get('name', '정보 없음')}"
        ).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.